In [ ]:
# shape of x is number of samples x time steps(lookback) x number of features (type of data)
# shape of y is number of expected outcome x number of features (type of data)

In [ ]:
# import libraries
import numpy as np

def create_sequence(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

In [ ]:
# import libraries
from sklearn.preprocessing import MinMaxScaler

def create_scaler(dataset):
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(dataset)
    return scaler, dataset

In [ ]:
# import libraries
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

def create_model(dataset, look_back=1, epochs=100, batch_size=1, verbose=0):
    X, Y = create_sequence(dataset, look_back)
    X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]))
    Y = np.reshape(Y, (Y.shape[0], Y.shape[1]))
    
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
    #model.add(Dropout(0.2))
    model.add(LSTM(units=50))
    model.add(Dense(X.shape[2]))  # Assuming single feature for simplicity
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X, Y, epochs=epochs, batch_size=batch_size, verbose=verbose)
    
    return model

In [ ]:
def create_input_data(dataset, look_back=1):
    dataset = np.array(dataset)
    dataset = dataset[-look_back:]
    seq = create_sequence(dataset, look_back)
    np.reshape(seq, (1, seq[0].shape[0], seq[0].shape[1]))
    return seq


In [ ]:
#
def predict(model, dataset, look_back=1):
    # take last look_back days from dataset
    dataset = dataset[-look_back:]
    # create input datas
    X = create_input_data(dataset, look_back)
    # make prediction
    prediction = model.predict(X)
    return prediction

In [ ]:
#
def predict_next_days(model, dataset, look_back=1, days=1):
    predictions = []
    for i in range(days):
        prediction = predict_next_day(model, dataset, look_back)
        predictions.append(prediction)
        dataset = np.append(dataset, prediction)
    return predictions